**Install Dependencies**

In [1]:
!pip install google-adk opentelemetry-instrumentation-google-genai

INFO: pip is looking at multiple versions of opentelemetry-semantic-conventions to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, bu

**Import & Set Project**

In [2]:
import os
from kaggle_secrets import UserSecretsClient

# Google Cloud credentials from Kaggle Add-on
user_secrets = UserSecretsClient()
creds = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(creds)

# Your GCP Project ID
PROJECT_ID = "civil-ivy-478206-s0"   # ⬅️ replace

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = "global"

print("Project set:", PROJECT_ID)


Project set: civil-ivy-478206-s0


**Create Project Folder**

In [3]:
!mkdir -p sos_agent

**requirements.txt**

In [4]:
%%writefile sos_agent/requirements.txt
google-adk
opentelemetry-instrumentation-google-genai


Writing sos_agent/requirements.txt


**.env**

In [5]:
%%writefile sos_agent/.env
GOOGLE_CLOUD_LOCATION="global"
GOOGLE_GENAI_USE_VERTEXAI=1


Writing sos_agent/.env


**.agent_engine_config.json**

In [8]:
%%writefile sos_agent/.agent_engine_config.json
{
    "min_instances": 0,
    "max_instances": 1,
    "resource_limits": {
        "cpu": "1",
        "memory": "1Gi"
    }
}


Writing sos_agent/.agent_engine_config.json


**agent.py (Women Safety Agent)**

In [9]:
%%writefile sos_agent/agent.py
from google.adk.agents import Agent
import vertexai
import os
import json
import time
from pathlib import Path

vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"]
)

OUT_DIR = Path("sos_demo_outputs")
OUT_DIR.mkdir(exist_ok=True)

HELP_KEYWORDS = [
    "help", "save", "danger", "unsafe", "kidnap", "attack",
    "bachao", "madad", "खतरा", "अपहरण", "हमला", "मदद", "सुरक्षित नहीं"
]

def now_str():
    return time.strftime("%Y%m%d_%H%M%S")

def save_json(obj, path):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

# -------- TOOLS --------

def transcribe_audio_tool(audio_file_url: str) -> dict:
    return {
        "status": "ok",
        "transcript": f"[Placeholder transcript for {audio_file_url}]",
        "audio_url": audio_file_url
    }

def detect_emergency_tool(text: str) -> dict:
    text_low = text.lower()
    matches = [w for w in HELP_KEYWORDS if w in text_low]

    if not matches:
        return {"is_emergency": False, "level": "none", "matches": []}

    level = "critical" if any(x in text_low for x in ["kidnap", "attack"]) else "danger"

    return {
        "is_emergency": True,
        "level": level,
        "matches": matches
    }

def confirm_sos_tool(prompt: str) -> dict:
    return {"confirmation_prompt": prompt}

def send_sos_simulated_tool(details: dict) -> dict:
    record = {
        "time": now_str(),
        "details": details,
        "note": "SOS simulated — no real dispatch."
    }
    file = OUT_DIR / f"sos_{now_str()}.json"
    save_json(record, file)
    return {"status": "ok", "saved_to": str(file)}

def memory_store_tool(event: dict) -> dict:
    file = OUT_DIR / "memory_log.json"
    if file.exists():
        data = json.load(open(file, "r"))
    else:
        data = []
    data.append(event)
    save_json(data, file)
    return {"status": "ok", "memory_file": str(file)}

# -------- AGENT --------

root_agent = Agent(
    name="women_safety_sos_agent",
    model="gemini-2.5-flash-lite",
    description="Multilingual SOS agent for women's safety.",
    instruction="""
You are a multilingual Women's Safety SOS Agent.

Rules:
- Understand ANY language.
- Always reply in the SAME language the user used.
- If danger is detected, call the detect_emergency_tool first.
- If an emergency exists → call confirm_sos_tool.
- After confirmation → call send_sos_simulated_tool.
- Store event using memory_store_tool.
- Be calming, short, supporting.
""",
    tools=[
        transcribe_audio_tool,
        detect_emergency_tool,
        confirm_sos_tool,
        send_sos_simulated_tool,
        memory_store_tool
    ]
)


Writing sos_agent/agent.py


**Local Demo**

In [10]:
from sos_agent.agent import detect_emergency_tool, confirm_sos_tool, send_sos_simulated_tool

txt = input("Type your message: ")

result = detect_emergency_tool(txt)
print("Emergency tool:", result)

if result["is_emergency"]:
    confirm = input("Emergency found. Type YES to send SOS: ")
    if confirm.lower() in ("yes", "y", "ha", "haan", "हाँ"):
        print(send_sos_simulated_tool({"reason": result}))
    else:
        print("Cancelled.")
else:
    print("No emergency detected.")


Type your message:  bachao


Emergency tool: {'is_emergency': True, 'level': 'danger', 'matches': ['bachao']}


Emergency found. Type YES to send SOS:  yes


{'status': 'ok', 'saved_to': 'sos_demo_outputs/sos_20251125_071746.json'}


**Deploy with ADK**

In [11]:
!adk deploy agent_engine \
  --project=$GOOGLE_CLOUD_PROJECT \
  --region=us-west1 \
  sos_agent \
  --agent_engine_config_file=sos_agent/.agent_engine_config.json


Staging all files in: /kaggle/working/sos_agent_tmp20251125_071823
Copying agent source code...
Copying agent source code complete.
Resolving files and dependencies...
Reading agent engine config from sos_agent/.agent_engine_config.json
Reading environment variables from /kaggle/working/sos_agent/.env
Ignoring GOOGLE_CLOUD_LOCATION in .env as `--region` was explicitly passed and takes precedence
Initializing Vertex AI...
Vertex AI initialized.
Created sos_agent_tmp20251125_071823/agent_engine_app.py
Files and dependencies resolved
Deploying to agent engine...
INFO:vertexai_genai.agentengines:Creating in-memory tarfile of source_packages
INFO:vertexai_genai.agentengines:Using agent framework: google-adk
INFO:vertexai_genai.agentengines:View progress and logs at https://console.cloud.google.com/logs/query?project=civil-ivy-478206-s0.
INFO:vertexai_genai.agentengines:Agent Engine created. To use it in another session:
INFO:vertexai_genai.agentengines:agent_engine=client.agent_engines.get(

**Test Agent**

In [12]:
import nest_asyncio
nest_asyncio.apply()

from vertexai import agent_engines
from vertexai.preview.generative_models import GenerativeModel
import vertexai
import os
import asyncio

# Force gRPC transport globally
vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location="us-west1",
    api_transport="grpc"   # ⭐ THIS FIXES STREAMING ISSUES ⭐
)

agents = list(agent_engines.list())
remote_agent = agents[0]

async def test_agent():
    generator = remote_agent.async_stream_query(
        message="Mujhe madad chahiye, koi follow kar raha hai.",
        user_id="demo_user"
    )
    async for chunk in generator:
        print(chunk)

await test_agent()





{'model_version': 'gemini-2.5-flash-lite', 'content': {'parts': [{'function_call': {'id': 'adk-74c16ffe-9753-49af-a130-a201c43bfa44', 'args': {'text': 'Mujhe madad chahiye, koi follow kar raha hai.'}, 'name': 'detect_emergency_tool'}}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 21, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 21}], 'prompt_token_count': 211, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 211}], 'total_token_count': 232, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.027363479137420654, 'invocation_id': 'e-dc3d9ff5-b09c-4704-8764-66b142b4dc9b', 'author': 'women_safety_sos_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'long_running_tool_ids': [], 'id': '2abb2488-bf3e-4b53-8daa-d0d3dd86106b', 'timestamp': 1764055565.888244}
{'content': {'parts': [{'function_response': {'id': 'adk-74c16ffe-9753-49af-a130-a201c

**Delete Agent**

In [13]:
from vertexai import agent_engines
agent_engines.delete(remote_agent.resource_name, force=True)


INFO:vertexai.agent_engines:Deleting AgentEngine resource: projects/831428492712/locations/us-west1/reasoningEngines/4967628718664056832
INFO:vertexai.agent_engines:Delete AgentEngine backing LRO: projects/831428492712/locations/us-west1/operations/2425950635774967808
INFO:vertexai.agent_engines:AgentEngine resource deleted: projects/831428492712/locations/us-west1/reasoningEngines/4967628718664056832
